# Cleaning and analysis of the attendance dataset

This notebook is more about illustrating the process of thinking, rather than doing the analysis the correct way or presenting some results.


In [2]:
import duckdb

# Week 1 - Data Cleaning with SQL and duckdb

Let's first extract a clean dataset that we can use in further analysis confidently.


The first step when facing some new data is to look at it. This is something that applies always, regardless of what problem you're facing - look at your data. Let's look at each table in turn.

Here are some best practices:
- It's always a good idea to look at a random sample of the data, rather than the first/last N rows.
- Begin with small tables first (in our case, `school` and `subject`) - tables that are about a single thing - rather than bigger tables that are a combination of multiple things.

## school

In [3]:
duckdb.sql(
    "SELECT * FROM 'attendance/school.parquet' ORDER BY random() LIMIT 10"
).show()

┌─────────────────┬─────────────┬───────────────┬─────────────────────────────────────────────────────────────────────┬───────────────┬───────────────────┬───────────────────┐
│    unique_id    │ school_code │ division_code │                             school_name                             │ division_name │ municipality_code │ municipality_name │
│     varchar     │    int64    │    varchar    │                               varchar                               │    varchar    │       int64       │      varchar      │
├─────────────────┼─────────────┼───────────────┼─────────────────────────────────────────────────────────────────────┼───────────────┼───────────────────┼───────────────────┤
│ c89c0f68d1153d7 │   190565420 │ NULL          │ Joniškio r. Kriukų pagrindinė mokykla                               │ NULL          │                47 │ Joniškio r. sav.  │
│ ac7f313918e5d7c │   191315590 │ NULL          │ Vilniaus r. Maišiagalos kun. Juzefo Obrembskio gimnazija            │ 

`school_code` should be the primary key in this table. Primary key is a column (or several columns) by which a unique row in the table is identified. By stating that `school_code` is a primary key, I explicitly expect two things from the column:
- This column has no duplicate values.
- This column has no null values.

Let's verify these assumptions.

In [ ]:
duckdb.sql(
    """
    SELECT 
        school_code,
        COUNT(*)
    FROM 'attendance/school.parquet' 
    GROUP BY school_code
    HAVING COUNT(*) > 1
"""
).show()

┌─────────────┬──────────────┐
│ school_code │ count_star() │
│    int64    │    int64     │
├─────────────┼──────────────┤
│   190066944 │            2 │
│   190986889 │            2 │
│   191130983 │            2 │
│   190341810 │            2 │
│   195170434 │            2 │
│   295093070 │            2 │
│   191791956 │            2 │
│   190341625 │            2 │
│   190697735 │            2 │
│   190892856 │            2 │
│   190189861 │            2 │
│   191090275 │            2 │
│   190545880 │            3 │
│   290893610 │            2 │
│   190696786 │            2 │
│   191130079 │            2 │
│   302843970 │            2 │
├─────────────┴──────────────┤
│ 17 rows          2 columns │
└────────────────────────────┘



We have duplicates :/ Let's look at a couple of them.

In [ ]:
duckdb.sql(
    """
    SELECT *
    FROM 'attendance/school.parquet'
    WHERE school_code IN (190341810, 191130983, 191130079)
"""
).show()

┌─────────────────┬─────────────┬───────────────┬─────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────┬───────────────────┬───────────────────┐
│    unique_id    │ school_code │ division_code │                 school_name                 │                            division_name                            │ municipality_code │ municipality_name │
│     varchar     │    int64    │    varchar    │                   varchar                   │                               varchar                               │       int64       │      varchar      │
├─────────────────┼─────────────┼───────────────┼─────────────────────────────────────────────┼─────────────────────────────────────────────────────────────────────┼───────────────────┼───────────────────┤
│ 33ba4eb90b8b20a │   190341810 │ 1446          │ Ukmergės Senamiesčio progimnazija           │ Ukmergės Senamiesčio progimnazijos Laičių pradinio ugdymo skyrius   │           

Looks like the school table is actually unique by school_code and division_code, but let's verify again

In [ ]:
duckdb.sql(
    """
    SELECT 
        school_code,
        division_code,
        COUNT(*)
    FROM 'attendance/school.parquet' 
    GROUP BY school_code, division_code
    HAVING COUNT(*) > 1
"""
).show()

┌─────────────┬───────────────┬──────────────┐
│ school_code │ division_code │ count_star() │
│    int64    │    varchar    │    int64     │
├─────────────┴───────────────┴──────────────┤
│                   0 rows                   │
└────────────────────────────────────────────┘



This is a data quality issue - it's unclear what division code NULL means, and the table documentation makes it seems that this is data on schools and not school divisions.

I'll deal with it in a somewhat silly way - I'll remove cases where `division_code` is not null. This is essentially saying - I won't look at schools that have divisions. Ideally we wouldn't do this, but out of concern for time, let's make this simplification. Also only 17 `school_codes` have the duplicate issue.

Simplifying in the beginning of an analysis is a good approach to manage complexity. We still don't know the dataset very well, so let's simplify. Once we are more familiar with the data, we can increase complexity and make our analysis more "correct".

Let's check if there are no null `school_codes`.


In [ ]:
duckdb.sql(
    """
    SELECT 
        count(distinct school_code),
        count(*)
    FROM 'attendance/school.parquet' 
    WHERE division_code IS NULL
"""
)

┌─────────────────────────────┬──────────────┐
│ count(DISTINCT school_code) │ count_star() │
│            int64            │    int64     │
├─────────────────────────────┼──────────────┤
│                         983 │          983 │
└─────────────────────────────┴──────────────┘

In [ ]:
# Final query for the school table
school = """
    SELECT 
        school_code, 
        school_name, 
        municipality_name
    FROM 'attendance/school.parquet' 
    WHERE division_code IS NULL
"""
duckdb.sql(school + " LIMIT 10").show()

┌─────────────┬─────────────────────────────────────────────────┬─────────────────────┐
│ school_code │                   school_name                   │  municipality_name  │
│    int64    │                     varchar                     │       varchar       │
├─────────────┼─────────────────────────────────────────────────┼─────────────────────┤
│   190545880 │ Biržų „Aušros“ pagrindinė mokykla               │ Biržų r. sav.       │
│   191130264 │ Plungės „Saulės“ gimnazija                      │ Plungės r. sav.     │
│   191814839 │ Mažeikių „Žiburėlio“ pradinė mokykla            │ Mažeikių r. sav.    │
│   190507118 │ Švenčionių r. Adutiškio pagrindinė mokykla      │ Švenčionių r. sav.  │
│   191317260 │ Vilniaus Trakų Vokės gimnazija                  │ Vilniaus m. sav.    │
│   191709681 │ Vilniaus Jono Ivaškevičiaus jaunimo mokykla     │ Vilniaus m. sav.    │
│   290486810 │ Vilkaviškio rajono Pajevonio pagrindinė mokykla │ Vilkaviškio r. sav. │
│   303283300 │ Panevėžio Raimun

## subject


In [ ]:
duckdb.sql(
    "SELECT * FROM 'attendance/subject.parquet' ORDER BY random() LIMIT 10"
).show()

┌─────────────────┬──────────────────┬──────────────┬───────────────────────────────────┬───────────────────────────────────────┐
│    unique_id    │ electronic_diary │ subject_code │           subject_name            │            subject_name_en            │
│     varchar     │     varchar      │    int64     │              varchar              │                varchar                │
├─────────────────┼──────────────────┼──────────────┼───────────────────────────────────┼───────────────────────────────────────┤
│ 3426c9d2cf672fb │ Tamo             │        24079 │ Piešimas                          │ Drawing                               │
│ a58a6c4e26d54f6 │ Tamo             │         4505 │ Užsienio kalba (antroji, ...)     │ Foreign language (second, ...)        │
│ 376ae967fae2520 │ NASC             │        24020 │ Instrumentuotė                    │ Instrumentation                       │
│ d9797b6c5143771 │ NASC             │        24007 │ Choro partitūrų skaitymas         │ 

This table should be unique by `electronic_diary` and `subject_code`, let's check.


In [ ]:
duckdb.sql(
    """
    SELECT
        electronic_diary,
        subject_code,
        COUNT(*)
    FROM 'attendance/subject.parquet'
    GROUP BY all
    HAVING COUNT(*) > 1
"""
).show()

┌──────────────────┬──────────────┬──────────────┐
│ electronic_diary │ subject_code │ count_star() │
│     varchar      │    int64     │    int64     │
├──────────────────┴──────────────┴──────────────┤
│                     0 rows                     │
└────────────────────────────────────────────────┘



In [ ]:
# Final query for the subject table
subject = """
    SELECT
        electronic_diary,
        subject_code,
        subject_name_en,
    FROM 'attendance/subject.parquet'
"""

yay!

## attendance

In [ ]:
duckdb.sql(
    "SELECT * FROM 'attendance/attendance.parquet' ORDER BY random() LIMIT 10"
).show()

┌─────────────────┬───────────────┬─────────────┬───────────────┬───────────────┬──────────────┬──────────────────┬───────────────────┬─────────────────────┬─────────────────────────┬───────────────────────┬───────────────────┐
│    unique_id    │ report_period │ school_code │ division_code │ student_class │ subject_code │ electronic_diary │ student_count_nsa │ student_count_diary │ excused_lessons_illness │ excused_lessons_other │ unexcused_lessons │
│     varchar     │     date      │    int64    │    varchar    │     int64     │    int64     │     varchar      │       int64       │        int64        │          int64          │         int64         │       int64       │
├─────────────────┼───────────────┼─────────────┼───────────────┼───────────────┼──────────────┼──────────────────┼───────────────────┼─────────────────────┼─────────────────────────┼───────────────────────┼───────────────────┤
│ 7dfe3db2bf81944 │ 2013-09-01    │   190134683 │ NULL          │             9 │       

- The rows in this table should be unique by `report_period`, `school_code`, `student_class` and `subject_code`. I should add `division_code` to this list, but I've already decided to not use that column. `electronic_diary` is not in this list because, if I remember correctly, each school uses just one of the diaries.
- `school_code` is the foreign key from the school table, while `subject_code` and `electronic_diary` are foreign keys, therefore I expect them to not have any NULL values.

Let's verify.


In [ ]:
duckdb.sql(
    """
    SELECT 
        report_period,
        school_code,
        student_class,
        subject_code,
        COUNT(*)
    FROM 'attendance/attendance.parquet'
    WHERE division_code IS NULL
    GROUP BY all
    HAVING COUNT(*) > 1
"""
).show()

┌───────────────┬─────────────┬───────────────┬──────────────┬──────────────┐
│ report_period │ school_code │ student_class │ subject_code │ count_star() │
│     date      │    int64    │     int64     │    int64     │    int64     │
├───────────────┼─────────────┼───────────────┼──────────────┼──────────────┤
│ 2024-04-01    │   191024624 │            10 │         6001 │            2 │
│ 2024-12-01    │   191024624 │            10 │         5301 │            2 │
│ 2024-05-01    │   190193030 │             9 │         8101 │            2 │
│ 2020-03-01    │   190046347 │            10 │         8101 │            2 │
│ 2024-04-01    │   300594972 │            11 │         4001 │            2 │
│ 2020-03-01    │   191316354 │             8 │         9101 │            2 │
│ 2024-02-01    │   190139463 │            10 │         5301 │            2 │
│ 2024-03-01    │   190189676 │            11 │         5301 │            2 │
│ 2019-10-01    │   190438615 │             5 │            1 │  

We have duplicates in the dataset again. But how many duplicates?

In [ ]:
duckdb.sql(
    """
    SELECT 
        report_period,
        school_code,
        student_class,
        subject_code,
        COUNT(*)
    FROM 'attendance/attendance.parquet'
    WHERE division_code IS NULL
    GROUP BY all
    HAVING COUNT(*) > 1
    ORDER BY random()
"""
).df().shape

(18776, 5)

Quite a few duplicated rows. 

Maybe I should have also included `electronic_diary` among the columns by which the attendance table is unique? This is weird, because why would a single class in a single school use two diaries? But weird things happen.

In [ ]:
duckdb.sql(
    """
    SELECT 
        report_period,
        school_code,
        division_code,
        student_class,
        subject_code,
        electronic_diary,
        COUNT(*)
    FROM 'attendance/attendance.parquet'
    WHERE division_code IS NULL
    GROUP BY all
    HAVING COUNT(*) > 1
    ORDER BY random()
"""
).df().shape

(12668, 7)

Ok, even grouping by `electronic_diary`, we have duplicates in the dataset. Let's look at several duplicated rows again.

In [ ]:
duckdb.sql(
    """
    SELECT 
        report_period,
        school_code,
        division_code,
        student_class,
        subject_code,
        electronic_diary,
        COUNT(*)
    FROM 'attendance/attendance.parquet'
    WHERE division_code IS NULL
    GROUP BY all
    HAVING COUNT(*) > 1
    ORDER BY random()
    LIMIT 5
"""
).show()

┌───────────────┬─────────────┬───────────────┬───────────────┬──────────────┬──────────────────┬──────────────┐
│ report_period │ school_code │ division_code │ student_class │ subject_code │ electronic_diary │ count_star() │
│     date      │    int64    │    varchar    │     int64     │    int64     │     varchar      │    int64     │
├───────────────┼─────────────┼───────────────┼───────────────┼──────────────┼──────────────────┼──────────────┤
│ 2024-01-01    │   190139278 │ NULL          │             8 │         8101 │ NASC             │            2 │
│ 2024-06-01    │   190244044 │ NULL          │            11 │         9102 │ NASC             │            2 │
│ 2024-05-01    │   190672543 │ NULL          │             7 │         8301 │ NASC             │            2 │
│ 2024-03-01    │   190533045 │ NULL          │             8 │         6001 │ NASC             │            2 │
│ 2024-11-01    │   190004615 │ NULL          │             6 │         3001 │ NASC             

In [ ]:
# Here I'm taking the first row from above and inserting the required values
duckdb.sql(
    """
    SELECT
        *
    FROM 'attendance/attendance.parquet'
    WHERE division_code IS NULL
    AND school_code = 290984490
    AND student_class = 1
    AND subject_code = 9101
    ORDER BY report_period
"""
)

┌─────────────────┬───────────────┬─────────────┬───────────────┬───────────────┬──────────────┬──────────────────┬───────────────────┬─────────────────────┬─────────────────────────┬───────────────────────┬───────────────────┐
│    unique_id    │ report_period │ school_code │ division_code │ student_class │ subject_code │ electronic_diary │ student_count_nsa │ student_count_diary │ excused_lessons_illness │ excused_lessons_other │ unexcused_lessons │
│     varchar     │     date      │    int64    │    varchar    │     int64     │    int64     │     varchar      │       int64       │        int64        │          int64          │         int64         │       int64       │
├─────────────────┼───────────────┼─────────────┼───────────────┼───────────────┼──────────────┼──────────────────┼───────────────────┼─────────────────────┼─────────────────────────┼───────────────────────┼───────────────────┤
│ 047c0a11c721ffe │ 2015-11-01    │   290984490 │ NULL          │             1 │       

It seems that in this case we just have duplicated rows. Here's what I'm noticing:
- for the duplicated rows, values in columns `student_count_diary` and `student_count_nsa` are the same.
- Values for the other columns (`excused_lessons_illness` etc.) are usually different. 

To solve the problem of duplicates in this table, I'll just aggregated by the columns that 

In [ ]:
attendance = """
    SELECT
        report_period,
        school_code,
        student_class,
        subject_code,
        electronic_diary,
        FIRST(student_count_diary) student_count_diary,
        SUM(excused_lessons_illness) excused_lessons_illness,
        SUM(excused_lessons_other) excused_lessons_other,
        SUM(unexcused_lessons) unexcused_lessons
    FROM 'attendance/attendance.parquet'
    WHERE division_code IS NULL
    GROUP BY all
"""
duckdb.sql(attendance + " LIMIT 10").show()

┌───────────────┬─────────────┬───────────────┬──────────────┬──────────────────┬─────────────────────┬─────────────────────────┬───────────────────────┬───────────────────┐
│ report_period │ school_code │ student_class │ subject_code │ electronic_diary │ student_count_diary │ excused_lessons_illness │ excused_lessons_other │ unexcused_lessons │
│     date      │    int64    │     int64     │    int64     │     varchar      │        int64        │         int128          │        int128         │      int128       │
├───────────────┼─────────────┼───────────────┼──────────────┼──────────────────┼─────────────────────┼─────────────────────────┼───────────────────────┼───────────────────┤
│ 2018-11-01    │   190007344 │             9 │         8101 │ NASC             │                  25 │                      13 │                     6 │                 0 │
│ 2016-04-01    │   191017430 │             5 │            1 │ NASC             │                  13 │                       0 │ 

I now have a dataset that I am sure is unique by `report_period`, `school_code`, `student_class` and `subject_code`, `electronic_diary` because that's how GROUP BY works.


In [ ]:
duckdb.sql(
    "SELECT * FROM 'attendance/attendance.parquet'"
    "WHERE report_period = '2019-01-01'"
    "AND school_code = 190001124"
    "AND student_class = 12"
).show()

┌─────────────────┬───────────────┬─────────────┬───────────────┬───────────────┬──────────────┬──────────────────┬───────────────────┬─────────────────────┬─────────────────────────┬───────────────────────┬───────────────────┐
│    unique_id    │ report_period │ school_code │ division_code │ student_class │ subject_code │ electronic_diary │ student_count_nsa │ student_count_diary │ excused_lessons_illness │ excused_lessons_other │ unexcused_lessons │
│     varchar     │     date      │    int64    │    varchar    │     int64     │    int64     │     varchar      │       int64       │        int64        │          int64          │         int64         │       int64       │
├─────────────────┼───────────────┼─────────────┼───────────────┼───────────────┼──────────────┼──────────────────┼───────────────────┼─────────────────────┼─────────────────────────┼───────────────────────┼───────────────────┤
│ 090e52480e60f23 │ 2019-01-01    │   190001124 │ NULL          │            12 │       

## final query

Ok, I think I'm ready to work on the final dataset now, so let's join everything together. I saved my queries into Python variables for a reason - now combining them is very easy. 

Keep in mind, that below I'm only using the WITH statement and inserting the tables separately because to all of them except the `subject` table I had to apply some sort of deduplication or grouping - logic that I now want to end up in the final query as well.

Also note that we're using GROUP BY to get rid of the `electronic_diary` column. There are some rows that have the same `school`, `subject`, `period` and `student_class`, but different `electronic_diary` values. I'm not sure what explains this, maybe it's schools transitioning from one diary to another. For my analysis, however, having the `electronic_diary` column is not important, therefore let's get rid of it.

Also note how big the final query is - that's a lot of logic! It would have been very tricky to such a query from the get go. That's why we went through this iterative process in the seminar.


In [5]:
attendance = """
    SELECT
        report_period,
        school_code,
        student_class,
        subject_code,
        electronic_diary,
        FIRST(student_count_diary) student_count_diary,
        SUM(excused_lessons_illness) excused_lessons_illness,
        SUM(excused_lessons_other) excused_lessons_other,
        SUM(unexcused_lessons) unexcused_lessons
    FROM 'attendance/attendance.parquet'
    WHERE division_code IS NULL
    GROUP BY all
"""

subject = """
    SELECT
        electronic_diary,
        subject_code,
        subject_name_en,
    FROM 'attendance/subject.parquet'
"""

school = """
    SELECT 
        school_code, 
        school_name, 
        municipality_name
    FROM 'attendance/school.parquet' 
    WHERE division_code IS NULL
"""


query = f"""
WITH attendance AS ({attendance}),
    school AS ({school}),
    subject AS ({subject})
SELECT
    school.school_name,
    school.school_code,
    school.municipality_name,
    attendance.student_class,
    attendance.report_period,
    subject.subject_name_en,
    FIRST(attendance.student_count_diary) AS student_count,
    SUM(attendance.excused_lessons_illness) AS excused_lessons_illness,
    SUM(attendance.excused_lessons_other) AS excused_lessons_other,
    SUM(attendance.unexcused_lessons) AS unexcused_lessons
FROM attendance
JOIN school ON attendance.school_code = school.school_code
JOIN subject ON attendance.subject_code = subject.subject_code
    AND attendance.electronic_diary = subject.electronic_diary
GROUP BY all
"""
print(query)


WITH attendance AS (
    SELECT
        report_period,
        school_code,
        student_class,
        subject_code,
        electronic_diary,
        FIRST(student_count_diary) student_count_diary,
        SUM(excused_lessons_illness) excused_lessons_illness,
        SUM(excused_lessons_other) excused_lessons_other,
        SUM(unexcused_lessons) unexcused_lessons
    FROM 'attendance/attendance.parquet'
    WHERE division_code IS NULL
    GROUP BY all
),
    school AS (
    SELECT 
        school_code, 
        school_name, 
        municipality_name
    FROM 'attendance/school.parquet' 
    WHERE division_code IS NULL
),
    subject AS (
    SELECT
        electronic_diary,
        subject_code,
        subject_name_en,
    FROM 'attendance/subject.parquet'
)
SELECT
    school.school_name,
    school.school_code,
    school.municipality_name,
    attendance.student_class,
    attendance.report_period,
    subject.subject_name_en,
    FIRST(attendance.student_count_diary) AS stu

In [6]:
duckdb.sql(query).df()

,school_name,school_code,municipality_name,student_class,report_period,subject_name_en,student_count,excused_lessons_illness,excused_lessons_other,unexcused_lessons
0,Kauno „Aušros“ gimnazija,190133777,Kauno m. sav.,11,2022-01-01,History,129,149.0,18.0,4.0
1,Elektrėnų sav. Semeliškių gimnazija,190664977,Elektrėnų sav.,10,2019-10-01,Biology,18,8.0,0.0,0.0
2,Kauno Simono Daukanto progimnazija,190136734,Kauno m. sav.,7,2021-04-01,Music,140,12.0,22.0,1.0
3,Kauno „Vyturio“ gimnazija,190136887,Kauno m. sav.,6,2022-04-01,Subject module,94,31.0,8.0,0.0
4,Ukmergės r. Deltuvos pagrindinė mokykla,190343829,Ukmergės r. sav.,8,2018-09-01,Fine arts,7,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
4912871,Klaipėdos licėjus,191832328,Klaipėdos m. sav.,7,2024-02-01,Geography,90,26.0,17.0,0.0
4912872,Visagino „Žiburio“ pagrindinė mokykla,193170510,Visagino sav.,7,2021-09-01,Ethical education (ethics),30,1.0,0.0,0.0
4912873,Kelmės r. Pakražančio gimnazija,190093788,Kelmės r. sav.,5,2023-09-01,Technologies,7,2.0,0.0,0.0
4912874,Kauno Senamiesčio progimnazija,191816085,Kauno m. sav.,4,2024-09-01,"Foreign language (first, English)",33,58.0,0.0,0.0
